# Análisis Completo de Cartera - Portfolio Analyzer System

## 1. Inventario de Módulos y Planificación

### Módulos Detectados y Funciones Públicas Relevantes

#### `portfolio_analyzer.data_loader` (PortfolioCompositionLoader - Principal)
**Funciones de Carga y Transformación:**
- `load_portfolio_daily_data(filename="portfolio_consolidado_completo.csv")` → bool
- `load_movements_data(filename="movements_report_2025-01-01_2025-09-08.csv")` → bool
- `load_benchmark_data(filename="precios_completos_ars.csv")` → bool
- `get_analysis_ready_data(include_benchmarks=False, max_benchmarks=10)` → tuple[DataFrame, DataFrame]

**Funciones de Acceso a Datos:**
- `get_portfolio_evolution()` → DataFrame (composición diaria completa)
- `get_portfolio_weights_matrix()` → DataFrame (matriz pesos × fecha)
- `get_current_composition(date=None)` → dict (pesos por activo)
- `get_total_value_series()` → Series (valor total diario)
- `get_summary_report()` → str

#### `portfolio_analyzer.risk_analysis` (RiskCalculator)
**Funciones de Métricas de Riesgo:**
- `calculate_basic_metrics(returns)` → dict (rendimiento, volatilidad, sharpe, drawdown, etc.)
- `calculate_advanced_metrics(returns, benchmark_returns=None)` → dict (alpha, beta, treynor, etc.)
- `analyze_portfolio_assets(df, asset_list, benchmark_asset=None)` → DataFrame (métricas por activo)

#### `portfolio_analyzer.visualization` (PortfolioVisualizer)
**Funciones de Visualización:**
- `plot_normalized_performance(df, assets, save_path=None)` → fig
- `plot_correlation_matrix(returns_data, save_path=None)` → fig
- `plot_drawdown_analysis(backtest_results, save_path=None)` → fig
- `plot_risk_return_scatter(strategies_data, save_path=None)` → fig
- `plot_rolling_metrics(returns_series, window=63, save_path=None)` → fig

#### `portfolio_analyzer.optimization` (PortfolioOptimizer)
**Funciones de Optimización:**
- `optimize_sharpe(max_weight=1.0, min_weight=0.0)` → dict
- `optimize_min_volatility(max_weight=1.0, min_weight=0.0)` → dict
- `generate_efficient_frontier(n_portfolios=100)` → DataFrame

#### `portfolio_analyzer.simulation` (MonteCarloSimulator)
**Funciones de Simulación:**
- `calculate_portfolio_returns(weights_dict, data_df)` → array
- `simulate_portfolio_paths(portfolio_returns, time_horizons, num_sims=10000, use_garch=True)` → dict
- `stress_test_scenarios(portfolio_returns, weights_dict=None)` → dict
- `generate_probability_analysis(simulation_results)` → dict

### Mapa Tarea → Función

| **Tarea** | **Módulo** | **Función Principal** |
|-----------|------------|----------------------|
| Carga datos cartera | `data_loader` | `PortfolioCompositionLoader.load_portfolio_daily_data()` |
| Serie valor total | `data_loader` | `get_total_value_series()` |
| Composición temporal | `data_loader` | `get_portfolio_weights_matrix()` |
| Métricas de riesgo | `risk_analysis` | `analyze_portfolio_assets()` |
| Visualización performance | `visualization` | `plot_normalized_performance()` |
| Análisis drawdown | `visualization` | `plot_drawdown_analysis()` |
| Optimización cartera | `optimization` | `optimize_sharpe()`, `optimize_min_volatility()` |
| Simulaciones Monte Carlo | `simulation` | `simulate_portfolio_paths()` |
| Comparación benchmarks | `data_loader` | `get_analysis_ready_data(include_benchmarks=True)` |

### Supuestos y Parámetros por Defecto Detectados

**Archivos de Datos:**
- Datos principales: `outputs/portfolio_consolidado_completo.csv` (separador `;`)
- Movimientos: `movements_report_2025-01-01_2025-09-08.csv` (separador `;`)
- Benchmarks: `outputs/precios_completos_ars.csv` (separador `;`)

**Parámetros por Defecto:**
- Tasa libre de riesgo: `0.04` (4% anual)
- Ventana métricas móviles: `63` días (~3 meses)
- Simulaciones Monte Carlo: `10,000` iteraciones
- Horizonte simulación: múltiples períodos en días
- Filtro peso mínimo: `0.001` (0.1%)

**Formato de Datos Esperado:**
- Fechas formato `YYYY-MM-DD`
- Columnas cartera: `instrumento`, `cantidad`, `precio`, `total`, `fecha`
- Nombres instrumentos en español completo

# 2. Setup e Imports desde Módulos del Repositorio

In [25]:
# Imports exclusivos desde módulos del repositorio
from portfolio_analyzer import (
    PortfolioCompositionLoader,
    RiskCalculator, 
    PortfolioVisualizer,
    PortfolioOptimizer,
    MonteCarloSimulator
)

# Librerías estándar necesarias
import pandas as pd
import numpy as np
import warnings

# Configuración básica
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

print("✅ Modules imported successfully")
print("📦 Portfolio Analyzer System - Composition Focus")

✅ Modules imported successfully
📦 Portfolio Analyzer System - Composition Focus


# 3. Carga y Validación de Datos

## Inicialización y Carga de Datos Principales

In [26]:
# Inicializar cargador principal
loader = PortfolioCompositionLoader()

# Cargar datos principales de la cartera
success_portfolio = loader.load_portfolio_daily_data()

if not success_portfolio:
    raise RuntimeError("❌ No se pudieron cargar los datos principales de la cartera")

💼 CARGANDO CARTERA REAL DIARIA...
✅ CARTERA DIARIA CARGADA:
   📊 Total registros: 292
   📅 Período: 2025-06-24 → 2025-09-07
   🎯 Instrumentos únicos: 8
🎯 INSTRUMENTOS EN TU CARTERA (resumen)


,instrumento,dias_presencia,primera_fecha,ultima_fecha,valor_total_acum
0,COCORMA,73,2025-06-24,2025-09-07,3186261
1,CEPU,69,2025-07-01,2025-09-07,1157625
2,AAPL,54,2025-07-16,2025-09-07,1589400
3,EWZ,26,2025-08-13,2025-09-07,499975
4,SPY,26,2025-08-13,2025-09-07,1124450
5,BHIP,15,2025-07-01,2025-07-15,166206
6,METR,15,2025-07-01,2025-07-15,290250
7,IBM,14,2025-07-30,2025-08-12,624100


📊 CALCULANDO COMPOSICIÓN DIARIA...
✅ Composición calculada para 73 días
💰 TOP PESOS PROMEDIO (>%0.1%)


,activo,peso_promedio,peso_promedio_pct
0,COCORMA,0.3841,38.41
1,AAPL,0.1797,17.97
2,CEPU,0.1330,13.30
3,SPY,0.1250,12.50
4,IBM,0.0703,7.03
5,EWZ,0.0555,5.55
6,METR,0.0333,3.33
7,BHIP,0.0191,1.91


In [27]:
# Intentar cargar datos auxiliares (opcionales)
print("📋 CARGANDO DATOS AUXILIARES...")

# Cargar movimientos (opcional)
success_movements = loader.load_movements_data()
if not success_movements:
    print("⚠️ Movimientos no cargados - continuando sin ellos")

# Cargar benchmarks (opcional) 
success_benchmarks = loader.load_benchmark_data()
if not success_benchmarks:
    print("⚠️ Benchmarks no cargados - continuando sin ellos")

📋 CARGANDO DATOS AUXILIARES...
✅ Movimientos cargados: 22 registros
💼 CALCULANDO POSICIONES REALES DESDE MOVIMIENTOS:
   ✅ POSICIONES REALES (ARS)


,symbol,valor_ars_actual,total_invertido,operaciones,ultima_fecha,is_cedear
0,SPY,42666.78,42666.78,1,2025-08-12,True
1,AAPL,26971.87,26971.87,1,2025-07-15,True
2,EWZ,18817.66,18817.66,1,2025-08-12,True
3,CEPU,5608.99,29085.34,2,2025-07-29,False
4,IBM,5399.60,46390.06,2,2025-08-12,True


📊 CARGANDO DATOS AUXILIARES PARA BENCHMARKS...
✅ Benchmarks auxiliares cargados:
   📊 Dimensiones: (2812, 42)
   📅 Período: 2020-01-02 → 2025-09-08
   🎯 Activos disponibles: 42


In [28]:
# Generar reporte de validación
validation_report = loader.get_summary_report()
print("\n🔍 VALIDACIÓN COMPLETADA")

RESUMEN DE DATOS CARGADOS - PORTFOLIO ANALYZER
💼 CARTERA REAL DIARIA: ✅
   • Registros: 292
   • Período: 2025-06-24 → 2025-09-07
   • Instrumentos: 8
   • Valor actual: $128,340 ARS
📋 MOVIMIENTOS REALES: ✅
   • Registros: 22
   • Posiciones calculadas: 5
📊 BENCHMARKS AUXILIARES: ✅
   • Activos: 42
   • Período: 2020-01-02 → 2025-09-08

🔍 VALIDACIÓN COMPLETADA


# 4. Análisis de Series de la Cartera

## Serie de Valor Total y Estadísticas Básicas

In [29]:
# Obtener serie de valor total
total_value_series = loader.get_total_value_series()

print("ESTADISTICAS DE VALOR TOTAL:")
print(f"   Valor inicial: ${total_value_series.iloc[0]:,.0f} ARS")
print(f"   Valor final: ${total_value_series.iloc[-1]:,.0f} ARS") 
print(f"   Valor máximo: ${total_value_series.max():,.0f} ARS")
print(f"   Valor mínimo: ${total_value_series.min():,.0f} ARS")

# Calcular retorno total
retorno_total = (total_value_series.iloc[-1] / total_value_series.iloc[0]) - 1
print(f"   Retorno total: {retorno_total:.1%}")

# Calcular retornos diarios
daily_returns = total_value_series.pct_change().dropna()
print(f"   Retorno diario promedio: {daily_returns.mean():.4f}")
print(f"   Volatilidad diaria: {daily_returns.std():.4f}")
print(f"   Días de análisis: {len(total_value_series)}")

# Crear DataFrame con estadísticas resumen
stats_summary = pd.DataFrame({
    'Métrica': ['Valor Inicial (ARS)', 'Valor Final (ARS)', 'Retorno Total', 
                'Volatilidad Diaria', 'Mejor Día', 'Peor Día', 'Días Positivos'],
    'Valor': [f"${total_value_series.iloc[0]:,.0f}",
              f"${total_value_series.iloc[-1]:,.0f}",
              f"{retorno_total:.1%}",
              f"{daily_returns.std():.2%}",
              f"{daily_returns.max():.2%}",
              f"{daily_returns.min():.2%}",
              f"{(daily_returns > 0).sum()}/{len(daily_returns)}"]
})

print("RESUMEN ESTADISTICO:")
display(stats_summary)

ESTADISTICAS DE VALOR TOTAL:
   Valor inicial: $79,927 ARS
   Valor final: $128,340 ARS
   Valor máximo: $128,340 ARS
   Valor mínimo: $59,847 ARS
   Retorno total: 60.6%
   Retorno diario promedio: 0.0121
   Volatilidad diaria: 0.1255
   Días de análisis: 73
RESUMEN ESTADISTICO:


,Métrica,Valor
0,Valor Inicial (ARS),"$79,927"
1,Valor Final (ARS),"$128,340"
2,Retorno Total,60.6%
3,Volatilidad Diaria,12.55%
4,Mejor Día,96.08%
5,Peor Día,-25.26%
6,Días Positivos,33/72


In [30]:
# Preparar datos para análisis de activos individuales
prices_data, returns_data = loader.get_analysis_ready_data(include_benchmarks=False)

print(f"DATOS PREPARADOS PARA ANALISIS:")
print(f"   Matriz de precios: {prices_data.shape}")
print(f"   Matriz de retornos: {returns_data.shape}")
print(f"   Período: {prices_data.index.min().date()} → {prices_data.index.max().date()}")

# Mostrar información de los activos
data_info = pd.DataFrame({
    'Activo': prices_data.columns,
    'Observaciones_Precios': prices_data.count(),
    'Precio_Inicial': prices_data.iloc[0],
    'Precio_Final': prices_data.iloc[-1],
    'Observaciones_Retornos': returns_data.count()
}).reset_index(drop=True)

print("\nINFORMACION DE ACTIVOS:")
display(data_info)

📊 DATOS PREPARADOS PARA ANÁLISIS:
   💼 Precios de cartera: (73, 8)
DATOS PREPARADOS PARA ANALISIS:
   Matriz de precios: (73, 8)
   Matriz de retornos: (25, 8)
   Período: 2025-06-24 → 2025-09-07

INFORMACION DE ACTIVOS:


,Activo,Observaciones_Precios,Precio_Inicial,Precio_Final,Observaciones_Retornos
0,AAPL,54,NaN,16625.00,25
1,BHIP,15,NaN,NaN,25
2,CEPU,69,NaN,1395.00,25
3,COCORMA,73,8394.71,9002.34,25
4,EWZ,26,NaN,20575.00,25
5,IBM,14,NaN,NaN,25
6,METR,15,NaN,NaN,25
7,SPY,26,NaN,44900.00,25


# 5. Composición de la Cartera en el Tiempo

## Evolución de Pesos y Composición

In [31]:
# Obtener matriz de pesos históricos
weights_matrix = loader.get_portfolio_weights_matrix()

print("ANALISIS DE COMPOSICION TEMPORAL:")
print(f"   Matriz de pesos: {weights_matrix.shape}")
print(f"   Activos únicos: {len(weights_matrix.columns)}")
print(f"   Fechas analizadas: {len(weights_matrix)}")

# Mostrar evolución de activos principales
print(f"\nPESOS PROMEDIO POR ACTIVO:")
pesos_promedio = weights_matrix.mean().sort_values(ascending=False)

# Crear DataFrame con pesos promedio
pesos_df = pd.DataFrame({
    'Activo': pesos_promedio.index,
    'Peso_Promedio': pesos_promedio.values,
    'Peso_Porcentaje': (pesos_promedio.values * 100).round(1)
}).reset_index(drop=True)

display(pesos_df.head(10))

# Estadísticas adicionales de concentración
print(f"\nESTADISTICAS DE CONCENTRACION:")
print(f"   Activos con peso > 1%: {(pesos_promedio > 0.01).sum()}")
print(f"   Activos con peso > 5%: {(pesos_promedio > 0.05).sum()}" )
print(f"   Peso del top 3: {pesos_promedio.head(3).sum():.1%}")

# ==============================================
# NUEVO: Performance semanal (tabla + gráfico)
# ==============================================
try:
    print("\n📆 CALCULANDO PERFORMANCE SEMANAL DEL PORTFOLIO Y ACTIVOS PRINCIPALES...")
    # Usar serie de valor total ya calculada anteriormente (total_value_series)
    if 'total_value_series' in globals():
        weekly_portfolio_value = total_value_series.resample('W').last()
        weekly_portfolio_return = weekly_portfolio_value.pct_change()

        weekly_table = pd.DataFrame({
            'Semana_Fin': weekly_portfolio_return.index.strftime('%Y-%m-%d'),
            'Retorno_Semanal_%': (weekly_portfolio_return * 100).round(2),
            'Valor_Cierre_Semana': weekly_portfolio_value.round(2).values
        }).dropna().reset_index(drop=True)

        print("\n📊 TABLA DE RETORNOS SEMANALES (últimas 10 semanas):")
        display(weekly_table.tail(10))
    else:
        print("⚠️ 'total_value_series' no disponible para cálculo semanal")

    # Performance semanal por activo (top 5 por peso promedio)
    if 'prices_data' in globals():
        weekly_prices = prices_data.resample('W').last()
        top_activos = pesos_promedio.head(5).index.tolist()

        # Instanciar visualizador si no existe
        vis = visualizer if 'visualizer' in globals() else PortfolioVisualizer()
        print(f"\n📈 GENERANDO GRAFICO DE PERFORMANCE SEMANAL NORMALIZADA (Top {len(top_activos)} Activos)...")
        fig_weekly = vis.plot_normalized_performance(
            df=weekly_prices,
            assets=top_activos,
            save_path="performance_semanal.png"
        )
    else:
        print("⚠️ 'prices_data' no disponible para gráfico semanal")
except Exception as e:
    print(f"❌ Error en cálculo/generación de performance semanal: {e}")

# ==============================================
# NUEVO: Valor TOTAL de la cartera (ARS) - Línea
# ==============================================
try:
    import plotly.graph_objects as go
    if 'total_value_series' in globals():
        print("\n💰 GRAFICANDO VALOR TOTAL DE LA CARTERA (ARS)...")
        tv = total_value_series.sort_index()
        fig_total = go.Figure()
        fig_total.add_trace(go.Scatter(
            x=tv.index,
            y=tv.values,
            mode='lines',
            name='Valor Cartera',
            line=dict(color='#2ecc71', width=3),
            hovertemplate='Fecha: %{x}<br>Valor: $%{y:,.0f} ARS<extra></extra>'
        ))
        # Línea punteada del valor inicial
        fig_total.add_hline(y=tv.iloc[0], line_dash='dot', line_color='gray',
                            annotation_text='Valor Inicial', annotation_position='top left')
        fig_total.update_layout(
            title='Valor Total de la Cartera (ARS)',
            xaxis_title='Fecha',
            yaxis_title='ARS',
            template='plotly_white',
            height=500,
            hovermode='x unified'
        )
        fig_total.show()
    else:
        print("⚠️ 'total_value_series' no disponible para graficar valor total")
except Exception as e:
    print(f"❌ Error generando gráfico de valor total: {e}")

ANALISIS DE COMPOSICION TEMPORAL:
   Matriz de pesos: (73, 8)
   Activos únicos: 8
   Fechas analizadas: 73

PESOS PROMEDIO POR ACTIVO:


,Activo,Peso_Promedio,Peso_Porcentaje
0,COCORMA,0.3841,38.4
1,AAPL,0.1797,18.0
2,CEPU,0.1330,13.3
3,SPY,0.1250,12.5
4,IBM,0.0703,7.0
5,EWZ,0.0555,5.6
6,METR,0.0333,3.3
7,BHIP,0.0191,1.9



ESTADISTICAS DE CONCENTRACION:
   Activos con peso > 1%: 8
   Activos con peso > 5%: 6
   Peso del top 3: 69.7%

📆 CALCULANDO PERFORMANCE SEMANAL DEL PORTFOLIO Y ACTIVOS PRINCIPALES...

📊 TABLA DE RETORNOS SEMANALES (últimas 10 semanas):


,Semana_Fin,Retorno_Semanal_%,Valor_Cierre_Semana
0,2025-07-06,49.91,120036.17
1,2025-07-13,-1.29,118488.30
2,2025-07-20,-0.59,117793.79
3,2025-07-27,2.05,120209.41
4,2025-08-03,2.31,122983.88
5,2025-08-10,0.80,123973.25
6,2025-08-17,-2.47,120914.80
7,2025-08-24,0.86,121957.81
8,2025-08-31,2.31,124775.24
9,2025-09-07,2.86,128340.43



📈 GENERANDO GRAFICO DE PERFORMANCE SEMANAL NORMALIZADA (Top 5 Activos)...
✅ Gráfico guardado como imagen en: c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\outputs\graficos\performance_semanal.png
✅ Gráfico guardado como imagen en: c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\outputs\graficos\performance_semanal.png
✅ Gráfico guardado como HTML interactivo en: c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\outputs\graficos\performance_semanal.html
✅ Gráfico guardado como HTML interactivo en: c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\outputs\graficos\performance_semanal.html



💰 GRAFICANDO VALOR TOTAL DE LA CARTERA (ARS)...


In [32]:
# Obtener composición actual
current_composition = loader.get_current_composition()


# Crear DataFrame con composición actual
current_comp_df = pd.DataFrame({
    'Activo': list(current_composition.keys()),
    'Peso': list(current_composition.values()),
    'Peso_Porcentaje': [f"{v:.1%}" for v in current_composition.values()]
}).sort_values('Peso', ascending=False).reset_index(drop=True)


💼 COMPOSICIÓN ACTUAL (pesos)


,activo,peso,peso_pct
0,SPY,0.3499,34.99
1,AAPL,0.2591,25.91
2,COCORMA,0.1764,17.64
3,EWZ,0.1603,16.03
4,CEPU,0.0543,5.43


In [33]:
# Analizar concentración temporal
print("ANALISIS DE CONCENTRACION:")
# Calcular índice de concentración (HHI) por fecha
hhi_series = (weights_matrix ** 2).sum(axis=1)
print(f"   HHI promedio: {hhi_series.mean():.3f}")
print(f"   HHI actual: {hhi_series.iloc[-1]:.3f}")
print(f"   Cartera más concentrada: {hhi_series.max():.3f} ({hhi_series.idxmax().date()})")
print(f"   Cartera más diversificada: {hhi_series.min():.3f} ({hhi_series.idxmin().date()})")

# Crear DataFrame con evolución de concentración
concentration_stats = pd.DataFrame({
    'Fecha': hhi_series.index,
    'HHI': hhi_series.values,
    'Num_Activos_Activos': (weights_matrix > 0.001).sum(axis=1).values,
    'Top3_Concentracion': weights_matrix.apply(lambda x: x.nlargest(3).sum(), axis=1).values
})

print("\nEVOLUCION DE CONCENTRACION (últimas 10 fechas):")
display(concentration_stats.tail(10))

ANALISIS DE CONCENTRACION:
   HHI promedio: 0.346
   HHI actual: 0.249
   Cartera más concentrada: 1.000 (2025-06-24)
   Cartera más diversificada: 0.244 (2025-08-13)

EVOLUCION DE CONCENTRACION (últimas 10 fechas):


,Fecha,HHI,Num_Activos_Activos,Top3_Concentracion
63,2025-08-29,0.2475,5,0.7826
64,2025-08-30,0.2473,5,0.7818
65,2025-08-31,0.2473,5,0.7818
66,2025-09-01,0.2473,5,0.7818
67,2025-09-02,0.2495,5,0.7836
68,2025-09-03,0.2482,5,0.7840
69,2025-09-04,0.2492,5,0.7876
70,2025-09-05,0.2494,5,0.7867
71,2025-09-06,0.2493,5,0.7853
72,2025-09-07,0.2493,5,0.7853


# 6. Comparación con Referencias y Benchmarks

In [34]:
# Comparación con benchmark personalizado usando BenchmarkBuilder
from portfolio_analyzer.benchmarks import BenchmarkBuilder

# Construir benchmark compuesto con configuración por defecto
print("🔨 Construyendo benchmark compuesto personalizado...")
builder = BenchmarkBuilder()
start_date = prices_data.index.min().strftime('%Y-%m-%d')
end_date = prices_data.index.max().strftime('%Y-%m-%d')
success_benchmark, bmk_package = builder.build_composite(
    start=start_date,
    end=end_date
)
if success_benchmark:
    component_prices = bmk_package['component_prices']
    composite_returns = bmk_package['composite_returns']
    composite_prices = bmk_package['composite_prices']
    print(f"📊 Benchmark compuesto construido: {len(component_prices.columns)} componentes, periodo {component_prices.index.min().date()} → {component_prices.index.max().date()}")
    display(composite_prices.to_frame(name='Benchmark Base 100').tail())
    benchmarks_available = True
else:
    print("⚠️ No se pudo construir el benchmark compuesto personalizado")
    benchmarks_available = False

🔨 Construyendo benchmark compuesto personalizado...

📊 Benchmark compuesto construido: 3 componentes, periodo 2025-06-24 → 2025-09-05
📊 Benchmark compuesto construido: 3 componentes, periodo 2025-06-24 → 2025-09-05


,Benchmark Base 100
Date,
2025-08-28,118.6351
2025-08-29,116.7583
2025-09-03,115.6411
2025-09-04,116.3847
2025-09-05,116.7230


In [35]:
if success_benchmark:
    print("\n🔬 COMPARANDO CARTERA VS. BENCHMARK COMPUESTO...")

    # 1. Obtener retornos de la cartera y del benchmark
    portfolio_returns = total_value_series.pct_change().dropna()
    portfolio_returns.name = 'Cartera'
    benchmark_returns = bmk_package['composite_returns']
    benchmark_returns.name = 'Benchmark'

    # 2. Combinar y alinear los retornos
    comparison_df = pd.concat([portfolio_returns, benchmark_returns], axis=1).dropna()

    # 3. Calcular métricas de riesgo comparativas
    risk_calc = RiskCalculator()
    
    # Métricas para la cartera y el benchmark
    portfolio_metrics = risk_calc.calculate_basic_metrics(comparison_df['Cartera'])
    portfolio_metrics['activo'] = 'Cartera'
    benchmark_metrics = risk_calc.calculate_basic_metrics(comparison_df['Benchmark'])
    benchmark_metrics['activo'] = 'Benchmark'

    # Métricas avanzadas (alfa, beta)
    advanced_metrics = risk_calc.calculate_advanced_metrics(
        returns=comparison_df['Cartera'],
        benchmark_returns=comparison_df['Benchmark']
    )
    if advanced_metrics:
        portfolio_metrics.update(advanced_metrics)
    # Asignar beta=1 y alpha=0 al benchmark (por definición)
    if 'beta' in portfolio_metrics:
        benchmark_metrics['beta'] = 1.0
    if 'alpha' in portfolio_metrics:
        benchmark_metrics['alpha'] = 0.0

    # 4. Crear tabla comparativa
    metrics_table = pd.DataFrame([portfolio_metrics, benchmark_metrics]).set_index('activo')
    
    display_cols = {
        'rendimiento_anual': 'Rendimiento Anual',
        'volatilidad': 'Volatilidad Anual',
        'sharpe_ratio': 'Sharpe Ratio',
        'max_drawdown': 'Max Drawdown',
        'alpha': 'Alpha',
        'beta': 'Beta'
    }
    available_cols = [col for col in display_cols.keys() if col in metrics_table.columns]
    
    print("\n📊 TABLA COMPARATIVA DE MÉTRICAS:")
    display(metrics_table[available_cols].rename(columns=display_cols))

    # 5. Graficar performance normalizada
    print("\n📈 GRÁFICO DE PERFORMANCE NORMALIZADA (CARTERA VS. BENCHMARK):")
    
    # Calcular precios normalizados (base 100)
    normalized_comparison = (1 + comparison_df).cumprod() * 100
    
    visualizer = PortfolioVisualizer()
    fig_comparison = visualizer.plot_normalized_performance(
        df=normalized_comparison,
        assets=['Cartera', 'Benchmark'],
        title='Performance Normalizada: Cartera vs. Benchmark',
        save_path="cartera_vs_benchmark"
    )
    fig_comparison.show()


🔬 COMPARANDO CARTERA VS. BENCHMARK COMPUESTO...
✅ RiskCalculator inicializado con risk_free_rate=0.00015873015873015873

📊 TABLA COMPARATIVA DE MÉTRICAS:


,Rendimiento Anual,Volatilidad Anual,Sharpe Ratio,Max Drawdown,Alpha,Beta
activo,,,,,,
Cartera,4.6712,2.5535,1.8137,-0.2526,3.5861,1.1832
Benchmark,0.9171,0.1980,4.4293,-0.0419,0.0000,1.0000



📈 GRÁFICO DE PERFORMANCE NORMALIZADA (CARTERA VS. BENCHMARK):
✅ Gráfico guardado como imagen en: c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\outputs\graficos\cartera_vs_benchmark.png
✅ Gráfico guardado como HTML interactivo en: c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\outputs\graficos\cartera_vs_benchmark.html
✅ Gráfico guardado como imagen en: c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\outputs\graficos\cartera_vs_benchmark.png
✅ Gráfico guardado como HTML interactivo en: c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\outputs\graficos\cartera_vs_benchmark.html


In [36]:
# Comparación semana por semana: Cartera vs Benchmark
import pandas as pd

# Resample semanal (de lunes a lunes, o ajustar según tus datos)
weekly_comparison = comparison_df.resample('W').apply(lambda x: (1 + x).prod() - 1)
weekly_comparison.index = weekly_comparison.index.strftime('%Y-%m-%d')

# Tabla resumen semanal
weekly_table = weekly_comparison.rename(columns={
    'Cartera': 'Retorno Cartera Semanal',
    'Benchmark': 'Retorno Benchmark Semanal'
})

print("\n📅 TABLA DE RETORNOS SEMANALES (CARTERA VS. BENCHMARK):")
display(weekly_table)

# Si quieres agregar métricas adicionales por semana, puedes hacerlo aquí.


📅 TABLA DE RETORNOS SEMANALES (CARTERA VS. BENCHMARK):


,Retorno Cartera Semanal,Retorno Benchmark Semanal
2025-06-29,0.0018,0.0176
2025-07-06,0.4834,0.0372
2025-07-13,-0.0194,0.0241
2025-07-20,-0.0105,0.0190
2025-07-27,0.0098,0.0047
2025-08-03,0.0323,0.0717
2025-08-10,0.0083,-0.0229
2025-08-17,-0.0138,-0.0095
2025-08-24,-0.0112,0.0102
2025-08-31,0.0223,0.0090


# 7. Métricas de Riesgo y Drawdowns

## Análisis de Riesgo por Activo

In [50]:
# Inicializar calculadora de riesgo
risk_calc = RiskCalculator()

# Obtener lista de activos de la cartera
asset_list = prices_data.columns.tolist()

print(f"CALCULANDO METRICAS DE RIESGO PARA {len(asset_list)} ACTIVOS...")

# Calcular métricas completas por activo
risk_metrics = risk_calc.analyze_portfolio_assets(
    df=prices_data,
    asset_list=asset_list,
    benchmark_asset=None
)

print(f"\nMETRICAS DE RIESGO CALCULADAS:")
print(f"   Activos analizados: {len(risk_metrics)}")
print(f"   Métricas por activo: {len(risk_metrics.columns) if not risk_metrics.empty else 0}")

if not risk_metrics.empty:
    print("\nMETRICAS DISPONIBLES:")
    metrics_info = pd.DataFrame({
        'Métrica': risk_metrics.columns,
        'Tipo_Dato': [str(risk_metrics[col].dtype) for col in risk_metrics.columns],
        'Valores_No_Nulos': [risk_metrics[col].count() for col in risk_metrics.columns]
    })
    display(metrics_info.head(15))

    # ==============================
    # Calcular métricas para la cartera total
    # ==============================
    print("\nCALCULANDO METRICAS DE RIESGO PARA EL TOTAL DE LA CARTERA...")
    # Usar retornos diarios de la cartera (total_value_series)
    cartera_returns = total_value_series.pct_change().dropna()
    cartera_metrics = risk_calc.calculate_basic_metrics(cartera_returns)

    # Si hay benchmark disponible, calcular tracking error, beta y alpha
    tracking_error = None
    if 'benchmark_returns' in globals():
        # Alinear fechas
        aligned = pd.concat([cartera_returns, benchmark_returns], axis=1).dropna()
        if aligned.shape[1] == 2:
            diff = aligned.iloc[:,0] - aligned.iloc[:,1]
            tracking_error = diff.std() * (252 ** 0.5)
            cartera_metrics['tracking_error'] = tracking_error
            # Calcular beta y alpha
            advanced_metrics = risk_calc.calculate_advanced_metrics(
                returns=aligned.iloc[:,0],
                benchmark_returns=aligned.iloc[:,1]
            )
            if advanced_metrics:
                cartera_metrics.update(advanced_metrics)
            print(f"Tracking Error anualizado: {tracking_error:.4f}")
            if advanced_metrics:
                if 'beta' in advanced_metrics:
                    print(f"Beta: {advanced_metrics['beta']:.4f}")
                if 'alpha' in advanced_metrics:
                    print(f"Alpha: {advanced_metrics['alpha']:.4f}")
    
    print("\nMETRICAS DE LA CARTERA TOTAL:")
    for k, v in cartera_metrics.items():
        print(f"   {k}: {v}")
    
    # Guardar métricas para la próxima celda
    cartera_metrics_total = cartera_metrics.copy()

✅ RiskCalculator inicializado con risk_free_rate=0.00015873015873015873
CALCULANDO METRICAS DE RIESGO PARA 8 ACTIVOS...
🔬 Analizando 8 activos...
   ✅ Métricas calculadas para 8 activos
📋 Progreso cálculo métricas


,activo,estado,obs,ret_anual_%,vol_%,sharpe,beta
0,AAPL,OK,54,104.75,24.60,4.095,0.50
1,BHIP,OK,15,45.87,30.75,1.361,-0.18
2,CEPU,OK,69,1.67,33.78,-0.069,0.02
3,COCORMA,OK,73,24.48,1.49,13.731,-0.02
4,EWZ,OK,26,95.46,25.07,3.649,1.18
5,IBM,OK,14,-149.23,21.09,-7.266,-1.08
6,METR,OK,15,94.67,54.84,1.653,-0.21
7,SPY,OK,26,53.89,17.28,2.887,1.04



METRICAS DE RIESGO CALCULADAS:
   Activos analizados: 8
   Métricas por activo: 23

METRICAS DISPONIBLES:


,Métrica,Tipo_Dato,Valores_No_Nulos
0,activo,object,8
1,observaciones,int64,8
2,rendimiento_anual,float64,8
3,volatilidad,float64,8
4,sharpe_ratio,float64,8
5,sortino_ratio,float64,8
6,max_drawdown,float64,8
7,var_95,float64,8
8,cvar_95,float64,8
9,alpha,float64,8



CALCULANDO METRICAS DE RIESGO PARA EL TOTAL DE LA CARTERA...
Tracking Error anualizado: 2.5434
Beta: 1.1832
Alpha: 3.5861

METRICAS DE LA CARTERA TOTAL:
   rendimiento_anual: 3.043826908176138
   volatilidad: 1.993004255472624
   sharpe_ratio: 1.5071853960812573
   sortino_ratio: 2.556100272173989
   var_95: -0.02107126702847118
   cvar_95: -0.15335543354356723
   max_drawdown: -0.25258855487146864
   skewness: 6.130703523688029
   kurtosis: 47.44243675242555
   observaciones: 72
   tracking_error: 2.5433740138132506
   beta: 1.1832218872977973
   alpha: 3.586061016489915
   kelly_criterion: 0.19137963695758445
   calmar_ratio: 18.493265314003796
   ulcer_index: 5.620538227693897


In [53]:
# ==============================
# EVOLUCIÓN DIARIA DEL DRAWDOWN DE LA CARTERA TOTAL
# ==============================
import pandas as pd
import plotly.graph_objects as go

# Calcular drawdown diario
rolling_max_diario = total_value_series.cummax()
drawdown_diario = (total_value_series - rolling_max_diario) / rolling_max_diario

# Crear DataFrame con evolución diaria de drawdown
drawdown_diario_df = pd.DataFrame({
    'Fecha': total_value_series.index.strftime('%Y-%m-%d'),
    'Valor_Cierre': total_value_series.values,
    'Drawdown_Diario': drawdown_diario.values
})

print("\nEVOLUCIÓN DIARIA DEL DRAWDOWN DE LA CARTERA TOTAL:")
display(drawdown_diario_df.tail(15))

# Gráfico de drawdown diario (verde oscuro mate)
fig_dd_diario = go.Figure()
fig_dd_diario.add_trace(go.Scatter(
    x=drawdown_diario_df['Fecha'],
    y=drawdown_diario_df['Drawdown_Diario'],
    mode='lines+markers',
    name='Drawdown Diario',
    line=dict(color='rgba(20,60,30,0.95)', width=2),
    fill='tozeroy',
    fillcolor='rgba(20,60,30,0.25)',
    marker=dict(color='rgba(20,60,30,0.95)'),
    hovertemplate='Fecha: %{x}<br>Drawdown: %{y:.1%}<extra></extra>'
) )
fig_dd_diario.update_layout(
    title='Evolución Diaria del Drawdown de la Cartera',
    xaxis_title='Fecha',
    yaxis_title='Drawdown',
    template='plotly_white',
    height=400,
    hovermode='x unified'
 )
fig_dd_diario.show()


EVOLUCIÓN DIARIA DEL DRAWDOWN DE LA CARTERA TOTAL:


,Fecha,Valor_Cierre,Drawdown_Diario
58,2025-08-24,121957.81,-0.0172
59,2025-08-25,121957.81,-0.0172
60,2025-08-26,123754.27,-0.0028
61,2025-08-27,123679.40,-0.0034
62,2025-08-28,124328.33,0.0000
63,2025-08-29,124677.48,0.0000
64,2025-08-30,124775.24,0.0000
65,2025-08-31,124775.24,0.0000
66,2025-09-01,124775.24,0.0000
67,2025-09-02,126643.56,0.0000


In [38]:
# Mostrar resumen de métricas principales
if not risk_metrics.empty:
    print("RESUMEN DE METRICAS PRINCIPALES:")
    
    # Seleccionar columnas principales disponibles
    main_cols = ['activo', 'rendimiento_anual', 'volatilidad', 'sharpe_ratio', 'max_drawdown']
    available_cols = [col for col in main_cols if col in risk_metrics.columns]
    
    if available_cols:
        display_metrics = risk_metrics[available_cols].head(10)
        print("METRICAS POR ACTIVO:")
        display(display_metrics)
    
    # Mostrar todas las métricas disponibles para la cartera
    print("\nTODAS LAS METRICAS DISPONIBLES (primeros 10 activos):")
    display(risk_metrics.head(10))
    
    # Estadísticas agregadas
    aggregate_stats = []
    if 'rendimiento_anual' in risk_metrics.columns:
        aggregate_stats.append(['Rendimiento Promedio', f"{risk_metrics['rendimiento_anual'].mean():.1%}"])
        
    if 'volatilidad' in risk_metrics.columns:
        aggregate_stats.append(['Volatilidad Promedio', f"{risk_metrics['volatilidad'].mean():.1%}"])
        
    if 'sharpe_ratio' in risk_metrics.columns:
        aggregate_stats.append(['Sharpe Ratio Promedio', f"{risk_metrics['sharpe_ratio'].mean():.3f}"])
        
    if 'max_drawdown' in risk_metrics.columns:
        aggregate_stats.append(['Drawdown Promedio', f"{risk_metrics['max_drawdown'].mean():.1%}"])
        
    if aggregate_stats:
        stats_df = pd.DataFrame(aggregate_stats, columns=['Estadística', 'Valor'])
        print("\nESTADISTICAS AGREGADAS:")
        display(stats_df)
        
    # Top y Bottom performers
    if 'rendimiento_anual' in risk_metrics.columns:
        top_performers = risk_metrics.nlargest(3, 'rendimiento_anual')[['activo', 'rendimiento_anual']]
        bottom_performers = risk_metrics.nsmallest(3, 'rendimiento_anual')[['activo', 'rendimiento_anual']]
        
        print("\nTOP 3 PERFORMERS:")
        display(top_performers)
        
        print("\nBOTTOM 3 PERFORMERS:")
        display(bottom_performers)
        
else:
    print("No se pudieron calcular métricas de riesgo")

RESUMEN DE METRICAS PRINCIPALES:
METRICAS POR ACTIVO:


,activo,rendimiento_anual,volatilidad,sharpe_ratio,max_drawdown
0,AAPL,1.0475,0.2460,4.0952,-0.0373
1,BHIP,0.4587,0.3075,1.3615,-0.0551
2,CEPU,0.0167,0.3378,-0.0691,-0.1977
3,COCORMA,0.2448,0.0149,13.7315,0.0000
4,EWZ,0.9546,0.2507,3.6486,-0.0403
5,IBM,-1.4923,0.2109,-7.2659,-0.0914
6,METR,0.9467,0.5484,1.6535,-0.0614
7,SPY,0.5389,0.1728,2.8870,-0.0239



TODAS LAS METRICAS DISPONIBLES (primeros 10 activos):


,activo,observaciones,rendimiento_anual,volatilidad,sharpe_ratio,sortino_ratio,max_drawdown,var_95,cvar_95,alpha,beta,r_squared,treynor_ratio,information_ratio,tracking_error,kelly_criterion,calmar_ratio,ulcer_index,fx_correlation,fx_beta,crisis_volatility_ratio,skewness,kurtosis
0,AAPL,53,1.0475,0.2460,4.0952,10.9100,-0.0373,-0.0144,-0.0179,1.0952,0.4990,7.4459e-02,2.0189,2.7571,0.2993,0.0255,28.0552,1.4053,None,None,None,2.1937,7.4482
1,BHIP,14,0.4587,0.3075,1.3615,1.7691,-0.0551,-0.0277,-0.0403,0.5421,-0.1778,6.4279e-03,-2.3555,-0.0312,0.3397,0.0000,8.3237,3.3980,None,None,None,-0.2025,1.2156
2,CEPU,68,0.0167,0.3378,-0.0691,-0.1032,-0.1977,-0.0377,-0.0469,1.0881,0.0180,9.6148e-05,-1.2973,1.6372,0.3413,0.0000,0.0844,9.5963,None,None,None,-0.1002,0.4116
3,COCORMA,72,0.2448,0.0149,13.7315,13.7315,0.0000,0.0000,0.0000,0.2565,-0.0219,6.8016e-02,-9.3383,-1.6970,0.1734,NaN,NaN,0.0000,None,None,None,1.3331,1.9639
4,EWZ,25,0.9546,0.2507,3.6486,5.1628,-0.0403,-0.0234,-0.0279,0.3208,1.1760,6.0575e-01,0.7777,2.6688,0.1558,0.0000,23.6738,1.5036,None,None,None,-0.0342,0.8117
5,IBM,13,-1.4923,0.2109,-7.2659,-11.9436,-0.0914,-0.0281,-0.0313,-0.8854,-1.0826,5.3153e-01,1.4154,-6.2885,0.3265,0.0000,-16.3268,4.1675,None,None,None,-0.6127,-0.3453
6,METR,14,0.9467,0.5484,1.6535,3.1049,-0.0614,-0.0400,-0.0530,1.0436,-0.2065,2.7296e-03,-4.3900,0.8604,0.5549,0.0000,15.4126,3.5270,None,None,None,0.9162,1.7922
7,SPY,25,0.5389,0.1728,2.8870,4.2104,-0.0239,-0.0083,-0.0161,-0.0225,1.0417,1.0000e+00,0.4790,NaN,0.0000,0.0000,22.5346,1.0535,None,None,None,0.6006,1.8893



ESTADISTICAS AGREGADAS:


,Estadística,Valor
0,Rendimiento Promedio,33.9%
1,Volatilidad Promedio,26.1%
2,Sharpe Ratio Promedio,2.505
3,Drawdown Promedio,-6.3%



TOP 3 PERFORMERS:


,activo,rendimiento_anual
0,AAPL,1.0475
4,EWZ,0.9546
6,METR,0.9467



BOTTOM 3 PERFORMERS:


,activo,rendimiento_anual
5,IBM,-1.4923
2,CEPU,0.0167
3,COCORMA,0.2448


# 8. Simulaciones Monte Carlo

## Simulación de Trayectorias Futuras

In [39]:
# Inicializar simulador Monte Carlo
mc_simulator = MonteCarloSimulator(risk_free_rate=0.04)

# Calcular retornos históricos de la cartera usando composición actual
portfolio_returns = mc_simulator.calculate_portfolio_returns(
    weights_dict=current_composition,
    data_df=prices_data
)

print(f"RETORNOS HISTORICOS DE CARTERA CALCULADOS:")
print(f"   Observaciones: {len(portfolio_returns)}")
print(f"   Retorno diario promedio: {np.mean(portfolio_returns):.4f}")
print(f"   Volatilidad diaria: {np.std(portfolio_returns):.4f}")

# Crear DataFrame con estadísticas de retornos
returns_stats = pd.DataFrame({
    'Estadística': ['Observaciones', 'Media Diaria', 'Desv. Estándar', 'Mínimo', 'Máximo', 
                    'Percentil 25', 'Mediana', 'Percentil 75', 'Asimetría', 'Curtosis'],
    'Valor': [
        len(portfolio_returns),
        f"{np.mean(portfolio_returns):.4f}",
        f"{np.std(portfolio_returns):.4f}",
        f"{np.min(portfolio_returns):.4f}",
        f"{np.max(portfolio_returns):.4f}",
        f"{np.percentile(portfolio_returns, 25):.4f}",
        f"{np.median(portfolio_returns):.4f}",
        f"{np.percentile(portfolio_returns, 75):.4f}",
        f"{pd.Series(portfolio_returns).skew():.3f}",
        f"{pd.Series(portfolio_returns).kurtosis():.3f}"
    ]
})

print("\nESTADISTICAS DE RETORNOS HISTORICOS:")
display(returns_stats)

RETORNOS HISTORICOS DE CARTERA CALCULADOS:
   Observaciones: 72
   Retorno diario promedio: 0.0024
   Volatilidad diaria: 0.0082

ESTADISTICAS DE RETORNOS HISTORICOS:


,Estadística,Valor
0,Observaciones,72
1,Media Diaria,0.0024
2,Desv. Estándar,0.0082
3,Mínimo,-0.0146
4,Máximo,0.0441
5,Percentil 25,0.0000
6,Mediana,0.0000
7,Percentil 75,0.0048
8,Asimetría,2.011
9,Curtosis,8.560


In [40]:
# Definir horizontes de simulación
time_horizons = [21, 63, 126, 252]  # 1, 3, 6 meses, 1 año

# Ejecutar simulaciones Monte Carlo
print(f"EJECUTANDO SIMULACIONES MONTE CARLO...")
simulation_results = mc_simulator.simulate_portfolio_paths(
    portfolio_returns=portfolio_returns,
    time_horizons=time_horizons,
    num_sims=1000,  # Reducido para performance
    use_garch=True
)

print(f"\nRESULTADOS DE SIMULACION:")

# Crear DataFrame con resultados de simulación
sim_results_data = []
for horizon_days, results in simulation_results.items():
    months = horizon_days // 21
    sim_results_data.append({
        'Horizonte_Días': horizon_days,
        'Horizonte_Meses': months,
        'Retorno_Esperado': f"{results['expected_return']:.1%}",
        'Volatilidad': f"{results['volatility']:.1%}",
        'Prob_Pérdida': f"{results['prob_loss']:.1%}",
        'Prob_Pérdida_10pct': f"{results['prob_loss_10']:.1%}",
        'Prob_Pérdida_20pct': f"{results['prob_loss_20']:.1%}",
        'Percentil_5': f"{results['percentiles'][5]:.1%}",
        'Percentil_50': f"{results['percentiles'][50]:.1%}",
        'Percentil_95': f"{results['percentiles'][95]:.1%}"
    })

sim_results_df = pd.DataFrame(sim_results_data)
print("\nTABLA DE SIMULACIONES:")
display(sim_results_df)

EJECUTANDO SIMULACIONES MONTE CARLO...
🔮 Simulando 1,000 trayectorias...
📊 ESTADÍSTICAS DIARIAS


,Retorno_Diario_%,Volatilidad_Diaria_%
0,0.12,0.824


⚙️ MODELO SELECCIONADO


,Modelo_Volatilidad
0,Modelo GARCH


🎯 HORIZONTES SIMULADOS


,Horizonte_Dias,Meses
0,21,1
1,63,3
2,126,6
3,252,12



RESULTADOS DE SIMULACION:

TABLA DE SIMULACIONES:


,Horizonte_Días,Horizonte_Meses,Retorno_Esperado,Volatilidad,Prob_Pérdida,Prob_Pérdida_10pct,Prob_Pérdida_20pct,Percentil_5,Percentil_50,Percentil_95
0,21,1,2.6%,5.1%,32.0%,0.2%,0.0%,-5.1%,2.5%,11.4%
1,63,3,7.9%,10.5%,22.2%,3.3%,0.1%,-8.4%,7.4%,26.0%
2,126,6,16.2%,17.0%,16.5%,4.8%,1.1%,-9.9%,15.9%,45.8%
3,252,12,34.2%,30.7%,10.9%,4.7%,1.5%,-8.9%,29.8%,92.9%


In [41]:
# Ejecutar stress testing
print(f"EJECUTANDO STRESS TESTING...")
stress_results = mc_simulator.stress_test_scenarios(
    portfolio_returns=portfolio_returns,
    weights_dict=current_composition
)

print(f"\nRESULTADOS DE STRESS TESTING:")

# Crear DataFrame con stress testing
stress_summary_data = []
for scenario, results in stress_results.items():
    stress_summary_data.append({
        'Escenario': scenario,
        'Probabilidad': f"{results['probability']:.1%}",
        'Duración_Días': results['duration_days'],
        'Retorno_Final': f"{results['final_return']:.1%}",
        'Max_Drawdown': f"{results['max_drawdown']:.1%}",
        'VaR_95': f"{results['var_95_stress']:.1%}"
    })

stress_summary_df = pd.DataFrame(stress_summary_data)
print("\nTABLA DE STRESS TESTING:")
display(stress_summary_df)

EJECUTANDO STRESS TESTING...
🎯 Ejecutando stress testing con escenarios argentinos...
🧪 RESULTADOS STRESS TEST


,Escenario,Retorno_Final_%,Max_DD_%,Duracion_Dias
0,Crisis Cambiaria,-28.4,-28.0,72
1,Devaluación Fuerte,-16.7,-19.2,60
2,Crisis Política,-20.8,-20.9,72
3,Recesión Local,-7.9,-9.4,72
4,Default Soberano,-35.2,-34.7,72



RESULTADOS DE STRESS TESTING:

TABLA DE STRESS TESTING:


,Escenario,Probabilidad,Duración_Días,Retorno_Final,Max_Drawdown,VaR_95
0,Crisis Cambiaria,15.0%,72,-28.4%,-28.0%,-1.7%
1,Devaluación Fuerte,20.0%,60,-16.7%,-19.2%,-1.7%
2,Crisis Política,10.0%,72,-20.8%,-20.9%,-1.5%
3,Recesión Local,25.0%,72,-7.9%,-9.4%,-1.3%
4,Default Soberano,8.0%,72,-35.2%,-34.7%,-1.8%


# 9. Verificaciones Finales y Auto-Chequeos

## Validación de Resultados y Consistencia

In [42]:
# Verificaciones de integridad de datos
print("VERIFICACIONES FINALES:")

# 1. Verificar dimensiones consistentes
print(f"\n1. DIMENSIONES DE DATOS:")
dimensions_check = pd.DataFrame({
    'Conjunto_Datos': ['Serie Valor Total', 'Matriz Pesos', 'Datos Precios', 'Retornos Diarios'],
    'Observaciones': [len(total_value_series), len(weights_matrix), len(prices_data), len(daily_returns)],
    'Columnas': ['N/A', weights_matrix.shape[1], prices_data.shape[1], returns_data.shape[1]]
})
display(dimensions_check)

# 2. Verificar rango temporal
print(f"\n2. CONSISTENCIA TEMPORAL:")
temporal_check = pd.DataFrame({
    'Conjunto': ['Precios', 'Pesos', 'Valor Total'],
    'Inicio': [prices_data.index.min().date(), weights_matrix.index.min().date(), total_value_series.index.min().date()],
    'Fin': [prices_data.index.max().date(), weights_matrix.index.max().date(), total_value_series.index.max().date()],
    'Días': [len(prices_data), len(weights_matrix), len(total_value_series)]
})
display(temporal_check)

# 3. Verificar suma de pesos
print(f"\n3. VALIDACION DE PESOS:")
peso_total_actual = sum(current_composition.values())
pesos_check = pd.DataFrame({
    'Validación': ['Suma Pesos Actuales', 'Activos con Posición', 'Peso Máximo Individual', 'Peso Mínimo > 0'],
    'Resultado': [f"{peso_total_actual:.1%}", len(current_composition), 
                 f"{max(current_composition.values()):.1%}", f"{min(current_composition.values()):.1%}"],
    'Status': ['OK' if abs(peso_total_actual - 1.0) < 0.01 else 'REVISAR', 'OK', 'OK', 'OK']
})
display(pesos_check)

VERIFICACIONES FINALES:

1. DIMENSIONES DE DATOS:


,Conjunto_Datos,Observaciones,Columnas
0,Serie Valor Total,73,N/A
1,Matriz Pesos,73,8
2,Datos Precios,73,8
3,Retornos Diarios,72,8



2. CONSISTENCIA TEMPORAL:


,Conjunto,Inicio,Fin,Días
0,Precios,2025-06-24,2025-09-07,73
1,Pesos,2025-06-24,2025-09-07,73
2,Valor Total,2025-06-24,2025-09-07,73



3. VALIDACION DE PESOS:


,Validación,Resultado,Status
0,Suma Pesos Actuales,100.0%,OK
1,Activos con Posición,5,OK
2,Peso Máximo Individual,35.0%,OK
3,Peso Mínimo > 0,5.4%,OK


# Visualizaciones y Análisis Gráfico

## Performance Normalizada de Activos

In [43]:
# Inicializar visualizador
visualizer = PortfolioVisualizer()

# Visualizar performance normalizada de activos principales
top_assets = pesos_promedio.head(5).index.tolist()
print("VISUALIZACION DE PERFORMANCE NORMALIZADA")
print(f"Activos principales: {len(top_assets)}")

# Crear gráfico de performance normalizada
fig_performance = visualizer.plot_normalized_performance(
    df=prices_data,
    assets=top_assets,
    save_path="performance_normalizada"
)

VISUALIZACION DE PERFORMANCE NORMALIZADA
Activos principales: 5
✅ Gráfico guardado como imagen en: c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\outputs\graficos\performance_normalizada.png
✅ Gráfico guardado como HTML interactivo en: c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\outputs\graficos\performance_normalizada.html
✅ Gráfico guardado como imagen en: c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\outputs\graficos\performance_normalizada.png
✅ Gráfico guardado como HTML interactivo en: c:\Users\trico\OneDrive\UBA\Management Financiero Bursatil\Cartera final\outputs\graficos\performance_normalizada.html


## Análisis de Optimización de Cartera

In [44]:
# Filtrar activos con varianza > 0
varianzas = returns_data.var()
activos_varianza = varianzas[varianzas > 0].index.tolist()
returns_data_filtrado = returns_data[activos_varianza]

# --- PARCHE: forzar mínimo de observaciones a 10 para el optimizador ---
from portfolio_analyzer import optimization
PortfolioOptimizer = optimization.PortfolioOptimizer
PortfolioOptimizer.__init__.__defaults__ = (None, 0.02)  # reset defaults

original_init = PortfolioOptimizer.__init__
def patched_init(self, returns_data, risk_free_rate=0.02):
    if self._is_price_data(returns_data):
        self.returns = returns_data.pct_change().dropna()
    else:
        self.returns = returns_data.dropna()
    self.returns = self.returns.replace([np.inf, -np.inf], np.nan).dropna()
    min_observations = 10
    valid_assets = []
    for col in self.returns.columns:
        non_null_count = self.returns[col].count()
        if non_null_count >= min_observations:
            valid_assets.append(col)
    if len(valid_assets) < 2:
        raise ValueError("Se necesitan al menos 2 activos con datos suficientes para optimización")
    self.returns = self.returns[valid_assets]
    self.assets = valid_assets
    self.n_assets = len(self.assets)
    self.rf_rate = risk_free_rate / 252
PortfolioOptimizer.__init__ = patched_init
# --- FIN PARCHE ---

try:
    optimizer = PortfolioOptimizer(returns_data=returns_data_filtrado, risk_free_rate=0.04)
    optimal_sharpe = optimizer.optimize_sharpe(max_weight=0.4, min_weight=0.0)
    if optimal_sharpe and optimal_sharpe['success']:
        print("\nCARTERA OPTIMA SHARPE:")
        print(f"Rendimiento esperado: {optimal_sharpe['rendimiento']:.1%}")
        print(f"Volatilidad: {optimal_sharpe['volatilidad']:.1%}")
        print(f"Sharpe Ratio: {optimal_sharpe['sharpe_ratio']:.3f}")
        sharpe_weights_df = pd.DataFrame(
            list(optimal_sharpe['weights'].items()),
            columns=['Activo', 'Peso_Optimal_Sharpe']
        ).sort_values('Peso_Optimal_Sharpe', ascending=False)
        print("\nPESOS OPTIMOS SHARPE:")
        display(sharpe_weights_df.head(10))
    else:
        print("No se pudo calcular cartera óptima Sharpe")
except Exception as e:
    print(f"❌ Error al inicializar optimizador o calcular Sharpe: {e}")


CARTERA OPTIMA SHARPE:
Rendimiento esperado: 66.3%
Volatilidad: 10.8%
Sharpe Ratio: 5.742

PESOS OPTIMOS SHARPE:


,Activo,Peso_Optimal_Sharpe
2,COCORMA,0.4
0,AAPL,0.4
3,EWZ,0.2
1,CEPU,0.0
4,SPY,0.0


In [45]:
# Optimización para mínima volatilidad
optimal_min_vol = optimizer.optimize_min_volatility(max_weight=0.4, min_weight=0.0)

if optimal_min_vol and optimal_min_vol['success']:
    print("\nCARTERA MINIMA VOLATILIDAD:")
    print(f"Rendimiento esperado: {optimal_min_vol['rendimiento']:.1%}")
    print(f"Volatilidad: {optimal_min_vol['volatilidad']:.1%}")
    print(f"Sharpe Ratio: {optimal_min_vol['sharpe_ratio']:.3f}")
    
    # Mostrar pesos óptimos
    min_vol_weights_df = pd.DataFrame(
        list(optimal_min_vol['weights'].items()),
        columns=['Activo', 'Peso_Min_Volatilidad']
    ).sort_values('Peso_Min_Volatilidad', ascending=False)
    
    print("\nPESOS MINIMA VOLATILIDAD:")
    display(min_vol_weights_df.head(10))
else:
    print("No se pudo calcular cartera mínima volatilidad")

# Comparar cartera actual vs óptimas
current_weights_df = pd.DataFrame(
    list(current_composition.items()),
    columns=['Activo', 'Peso_Actual']
).sort_values('Peso_Actual', ascending=False)

print("\nCOMPARACION DE COMPOSICIONES:")
print("CARTERA ACTUAL:")
display(current_weights_df.head(10))


CARTERA MINIMA VOLATILIDAD:
Rendimiento esperado: 23.6%
Volatilidad: 8.4%
Sharpe Ratio: 2.332

PESOS MINIMA VOLATILIDAD:

PESOS MINIMA VOLATILIDAD:


,Activo,Peso_Min_Volatilidad
2,COCORMA,4.0000e-01
4,SPY,2.7828e-01
0,AAPL,2.0503e-01
1,CEPU,1.1669e-01
3,EWZ,1.4519e-16



COMPARACION DE COMPOSICIONES:
CARTERA ACTUAL:


,Activo,Peso_Actual
4,SPY,0.3499
0,AAPL,0.2591
2,COCORMA,0.1764
3,EWZ,0.1603
1,CEPU,0.0543


In [46]:
# Generar frontera eficiente
try:
    efficient_frontier = optimizer.generate_efficient_frontier(n_portfolios=50)
    
    if not efficient_frontier.empty:
        print("\nFRONTERA EFICIENTE GENERADA:")
        print(f"Carteras calculadas: {len(efficient_frontier)}")
        
        # DEBUG: Mostrar columnas y primeras filas
        print("Columnas de efficient_frontier:", efficient_frontier.columns.tolist())
        print("Primeras filas de efficient_frontier:")
        print(efficient_frontier.head(3))
        
        # Mostrar algunas carteras de la frontera
        required_cols = ['rendimiento', 'volatilidad', 'sharpe_ratio']
        available_cols = [col for col in required_cols if col in efficient_frontier.columns]
        if len(available_cols) < 3:
            print(f"⚠️ Columnas faltantes en frontera eficiente: {set(required_cols) - set(available_cols)}")
            print(f"Columnas disponibles: {efficient_frontier.columns.tolist()}")
        
        if all(col in efficient_frontier.columns for col in required_cols):
            frontier_sample = efficient_frontier[required_cols].head(10)
            print("\nMUESTRA DE FRONTERA EFICIENTE:")
            display(frontier_sample)
        else:
            print("No se puede mostrar muestra de frontera eficiente por columnas faltantes.")
        
        # Crear scatter plot riesgo-retorno si hay datos suficientes
        if len(efficient_frontier) > 5 and all(col in efficient_frontier.columns for col in required_cols):
            print("\n📊 Creando gráfico riesgo-retorno...")
            strategies_data = []
            for idx, row in efficient_frontier.head(20).iterrows():
                strategies_data.append({
                    'nombre': f'Portfolio_{idx}',
                    'rendimiento': row['rendimiento'],
                    'riesgo': row['volatilidad'],
                    'sharpe': row.get('sharpe_ratio', None)
                })
            fig_scatter = visualizer.plot_risk_return_scatter(
                strategies_data=strategies_data,
                save_path="frontera_eficiente"
            )
        elif len(efficient_frontier) > 5:
            print("No se puede crear gráfico riesgo-retorno por columnas faltantes.")
    else:
        print("No se pudo generar frontera eficiente")
        
except Exception as e:
    print(f"Error generando frontera eficiente: {e}")


FRONTERA EFICIENTE GENERADA:
Carteras calculadas: 50
Columnas de efficient_frontier: ['rendimiento', 'volatilidad', 'sharpe_ratio', 'pesos']
Primeras filas de efficient_frontier:
   rendimiento  volatilidad  sharpe_ratio  \
0       0.5732       0.1091        4.8884   
1       0.2277       0.0816        2.2989   
2      -0.3148       0.1496       -2.3721   

                                               pesos  
0  [0.3158915516801789, 0.02642953699227935, 0.34...  
1  [0.1400921317620013, 0.11357872274914957, 0.43...  
2  [0.06087015993463707, 0.3643325351970654, 0.21...  

MUESTRA DE FRONTERA EFICIENTE:


,rendimiento,volatilidad,sharpe_ratio
0,0.5732,0.1091,4.8884
1,0.2277,0.0816,2.2989
2,-0.3148,0.1496,-2.3721
3,0.4493,0.1342,3.0492
4,0.4429,0.0958,4.2062
5,-0.0458,0.1566,-0.5477
6,0.2062,0.1403,1.1851
7,0.6299,0.2014,2.9299
8,-0.3635,0.2026,-1.9918
9,-0.0398,0.1526,-0.5225



📊 Creando gráfico riesgo-retorno...
📊 Creando gráfico riesgo-retorno...
Error generando frontera eficiente: 'sharpe_ratio'


## Análisis Probabilístico Detallado

In [47]:
# Generar análisis probabilístico detallado
probability_analysis = mc_simulator.generate_probability_analysis(simulation_results)

print("ANALISIS PROBABILISTICO DETALLADO")
print(f"Horizontes analizados: {len(probability_analysis)}")

# Crear DataFrame con análisis probabilístico
prob_data = []
for horizon, analysis in probability_analysis.items():
    prob_data.append({
        'Horizonte': horizon,
        'Dias': analysis['horizon_days'],
        'Meses': analysis['horizon_months'],
        'Retorno_Esperado': f"{analysis['expected_return']:.1%}",
        'Volatilidad': f"{analysis['volatility']:.1%}",
        'Prob_Positivo': f"{analysis['prob_positive']:.1%}",
        'Prob_Perdida': f"{analysis['prob_loss']:.1%}",
        'Prob_Perdida_10pct': f"{analysis['prob_loss_10']:.1%}",
        'Prob_Ganancia_20pct': f"{analysis['prob_gain_20']:.1%}",
        'Mejor_Caso_5pct': f"{analysis['best_case_5']:.1%}",
        'Peor_Caso_5pct': f"{analysis['worst_case_5']:.1%}",
        'Retorno_Mediano': f"{analysis['median_return']:.1%}"
    })

probability_df = pd.DataFrame(prob_data)

print("\nTABLA DE PROBABILIDADES POR HORIZONTE:")
display(probability_df)

📊 Generando análisis probabilístico...
ANALISIS PROBABILISTICO DETALLADO
Horizontes analizados: 4

TABLA DE PROBABILIDADES POR HORIZONTE:


,Horizonte,Dias,Meses,Retorno_Esperado,Volatilidad,Prob_Positivo,Prob_Perdida,Prob_Perdida_10pct,Prob_Ganancia_20pct,Mejor_Caso_5pct,Peor_Caso_5pct,Retorno_Mediano
0,1M,21,1,2.6%,5.1%,68.0%,32.0%,0.2%,0.1%,11.4%,-5.1%,2.5%
1,3M,63,3,7.9%,10.5%,77.8%,22.2%,3.3%,12.4%,26.0%,-8.4%,7.4%
2,6M,126,6,16.2%,17.0%,83.5%,16.5%,4.8%,39.3%,45.8%,-9.9%,15.9%
3,12M,252,12,34.2%,30.7%,89.1%,10.9%,4.7%,64.4%,92.9%,-8.9%,29.8%


In [48]:
# Crear DataFrame con resultados de stress testing
stress_data = []
for scenario, results in stress_results.items():
    stress_data.append({
        'Escenario': scenario,
        'Probabilidad': f"{results['probability']:.1%}",
        'Duracion_Dias': results['duration_days'],
        'Shock_Aplicado': f"{results['shock_applied']:.1%}",
        'Retorno_Final': f"{results['final_return']:.1%}",
        'Max_Drawdown': f"{results['max_drawdown']:.1%}",
        'VaR_95_Stress': f"{results['var_95_stress']:.1%}"
    })

stress_df = pd.DataFrame(stress_data)

print("\nTABLA DE STRESS TESTING:")
display(stress_df)

# Generar resumen completo de simulaciones
print("\n📋 Creando resumen de simulaciones...")
try:
    simulation_summary = mc_simulator.create_simulation_summary(
        simulation_results, 
        stress_results
    )
    # Mostrar el resumen solo si no está vacío
    if simulation_summary is not None and not simulation_summary.empty:
        print("\nRESUMEN DE SIMULACIONES GENERADO")
        print("Incluye análisis de todos los horizontes y escenarios")
    
except Exception as e:
    print(f"Advertencia: No se pudo generar resumen completo: {e}")



TABLA DE STRESS TESTING:


,Escenario,Probabilidad,Duracion_Dias,Shock_Aplicado,Retorno_Final,Max_Drawdown,VaR_95_Stress
0,Crisis Cambiaria,15.0%,72,-50.0%,-28.4%,-28.0%,-1.7%
1,Devaluación Fuerte,20.0%,60,-35.0%,-16.7%,-19.2%,-1.7%
2,Crisis Política,10.0%,72,-40.0%,-20.8%,-20.9%,-1.5%
3,Recesión Local,25.0%,72,-25.0%,-7.9%,-9.4%,-1.3%
4,Default Soberano,8.0%,72,-60.0%,-35.2%,-34.7%,-1.8%



📋 Creando resumen de simulaciones...
📋 Creando resumen de simulaciones...

RESUMEN DE SIMULACIONES GENERADO
Incluye análisis de todos los horizontes y escenarios
